# Keyword search on encrypted datasets using AES

In this netbook, a simple protocol for searching for keywords in Persian texts encrypted with AES has been simulated and tested. This netbook only includes the test implementation and the process is continued on real datasets. In this netbook, the general approach of the algorithms can be seen.

## Main libs

In [1]:
import hashlib
import hmac
import secrets
import pandas as pd
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

## Setup

The system is initialized with the security parameter 𝜆, generating public parameters 𝑝𝑎𝑟.

In [2]:
def setup():
    """
    System setup with Diffie–Hellman parameters
    """
    # 2048-bit MODP group (simplified demo)
    p = int(
        "FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD1"
        "29024E088A67CC74020BBEA63B139B22514A08798E3404DD"
        "EF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245"
        "E485B576625E7EC6F44C42E9A63A3620FFFFFFFFFFFFFFFF",
        16,
    )
    g = 2

    return {"p": p, "g": g}


## Diffie-Hellman Key exchange protocol

In [3]:
def diffie_hellman_key(par):
    p, g = par["p"], par["g"]

    # private keys
    a = secrets.randbelow(p)
    b = secrets.randbelow(p)

    # public keys
    A = pow(g, a, p)
    B = pow(g, b, p)

    # shared secret
    s1 = pow(B, a, p)
    s2 = pow(A, b, p)

    assert s1 == s2

    # derive symmetric key
    k = hashlib.sha256(str(s1).encode()).digest()

    return k


## AES scheme

𝐸𝑛𝑐(𝑚, 𝑘, 𝑤) → (𝐶𝑚, 𝐼, 𝑁)
The data owner performs encryption using the plaintext 𝑚 and the symmetric key 𝑘. The keyword 𝑤 is also encrypted to generate the index 𝐼. The plaintext 𝑚 and key 𝑘 are encrypted to produce the ciphertext 𝐶𝑚, and the file identifier 𝑁 is generated using symmetric encryption for identification. The system computes a hash value 𝐻 for 𝑁 and 𝐶𝑚, producing the final encrypted result 𝐶𝑇.

In [4]:
def aes_encrypt(key, plaintext):
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)
    return cipher.nonce, ciphertext, tag


def aes_decrypt(key, nonce, ciphertext, tag):
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    return cipher.decrypt_and_verify(ciphertext, tag)


def H(data: bytes) -> bytes:
    return hashlib.sha256(data).digest()


def keyword_enc(key, keyword: str) -> bytes:
    return hmac.new(key, keyword.encode("utf-8"), hashlib.sha256).digest()


## Multiple keyword encryption

In this study, each input has multiple keywords. So we have to encrypt all of them for each input.

In [5]:
def encrypt_document_multi(m: str, k: bytes, keywords: list[str]):
    """
    Encrypt document with multiple keyword indexes
    """
    # File identifier
    N = secrets.token_bytes(16)

    # Encrypt message
    nonce, Cm, tag = aes_encrypt(k, m.encode("utf-8"))

    # Multiple encrypted keyword indexes
    I_set = [keyword_enc(k, w) for w in keywords]

    # Hash for verification
    H_val = H(N + Cm)

    CT = {
        "Cm": Cm,
        "nonce": nonce,
        "tag": tag,
        "N": N,
        "H": H_val,
    }

    return CT, I_set


## Trapdoor

𝑇𝑤(𝑤𝑖) → 𝑇𝑤 The user computes a trapdoor 𝑇𝑤 by encrypting the keyword 𝑤𝑖. This trapdoor is then sent to the smart contract.

In [6]:
def trapdoor(k: bytes, w_i: str) -> bytes:
    return keyword_enc(k, w_i)


## Search

𝑠𝑒𝑎𝑟𝑐ℎ(𝑇𝑤, 𝐼) → (𝑘, 𝑁, 𝐻) The smart contract performs the search after receiving the trapdoor 𝑇𝑤 and index 𝐼. If the search is successful, it retrieves the symmetric encryption key k, the file identifier N, and the hash value 𝐻.

In [7]:
def search_multi(trapdoor, index_set):
    """
    OR-search over encrypted keyword indexes
    """
    for I in index_set:
        if hmac.compare_digest(trapdoor, I):
            return True
    return False


## Verification

𝑣𝑒𝑟𝑖𝑓𝑦(𝐶𝑇, 𝑁) → {0, 1} Upon receiving the ciphertext 𝐶𝑇 from the cloud server, the user verifies whether the server has correctly provided the data and whether the ciphertext has been tampered with. The user checks whether the file identifier 𝑁 and hash 𝐻 match those from the smart contract. If they do, the verification succeeds, returning 1; otherwise, it returns 0.

In [8]:
def verify(CT):
    recomputed_H = H(CT["N"] + CT["Cm"])
    return int(recomputed_H == CT["H"])


## Sample dataset for testing funcions

In [9]:
data = {
    "text": [
        "امنیت اطلاعات و حریم خصوصی بسیار مهم است",
        "رمزنگاری داده‌ها در فضای ابری و بلاکچین",
        "جستجوی امن کلمات کلیدی و قرارداد هوشمند",
        "بلاکچین و قرارداد هوشمند برای امنیت",
        "حریم خصوصی کاربران در سیستم‌های توزیع‌شده",
    ],
    "keywords": [
        ["امنیت", "حریم"],
        ["رمزنگاری", "بلاکچین"],
        ["جستجو", "قرارداد"],
        ["بلاکچین", "امنیت"],
        ["حریم", "خصوصی"],
    ],
}

df = pd.DataFrame(data)
df


,text,keywords
0,امنیت اطلاعات و حریم خصوصی بسیار مهم است,"[امنیت, حریم]"
1,رمزنگاری داده‌ها در فضای ابری و بلاکچین,"[رمزنگاری, بلاکچین]"
2,جستجوی امن کلمات کلیدی و قرارداد هوشمند,"[جستجو, قرارداد]"
3,بلاکچین و قرارداد هوشمند برای امنیت,"[بلاکچین, امنیت]"
4,حریم خصوصی کاربران در سیستم‌های توزیع‌شده,"[حریم, خصوصی]"


## Preparation

In [10]:
par = setup()
k = diffie_hellman_key(par)

encrypted_db = []

for _, row in df.iterrows():
    CT, I_set = encrypt_document_multi(
        row["text"], k, row["keywords"]
    )
    encrypted_db.append({"CT": CT, "I_set": I_set})


## Test

In [11]:
query = "بلاکچین"
Tw = trapdoor(k, query)

results = []

for entry in encrypted_db:
    if search_multi(Tw, entry["I_set"]):
        if verify(entry["CT"]):
            plaintext = aes_decrypt(
                k,
                entry["CT"]["nonce"],
                entry["CT"]["Cm"],
                entry["CT"]["tag"],
            ).decode("utf-8")
            results.append(plaintext)

results


['رمزنگاری داده\u200cها در فضای ابری و بلاکچین',
 'بلاکچین و قرارداد هوشمند برای امنیت']

In [12]:
query = "جستجو"
Tw = trapdoor(k, query)

results = []

for entry in encrypted_db:
    if search_multi(Tw, entry["I_set"]):
        if verify(entry["CT"]):
            plaintext = aes_decrypt(
                k,
                entry["CT"]["nonce"],
                entry["CT"]["Cm"],
                entry["CT"]["tag"],
            ).decode("utf-8")
            results.append(plaintext)

results


['جستجوی امن کلمات کلیدی و قرارداد هوشمند']

In [13]:
query = "پیدانکن"
Tw = trapdoor(k, query)

results = []

for entry in encrypted_db:
    if search_multi(Tw, entry["I_set"]):
        if verify(entry["CT"]):
            plaintext = aes_decrypt(
                k,
                entry["CT"]["nonce"],
                entry["CT"]["Cm"],
                entry["CT"]["tag"],
            ).decode("utf-8")
            results.append(plaintext)

results


[]